In [6]:
! pip install gensim
! pip install beautifulsoup4
! pip install wordcloud
! pip install tqdm

# или используем https://www.lfd.uci.edu/~gohlke/pythonlibs/#wordcloud

  Using cached wordcloud-1.3.1.tar.gz
  Running setup.py bdist_wheel for wordcloud: started
  Running setup.py bdist_wheel for wordcloud: finished with status 'error'
  Complete output from command C:\Users\akrazer\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\akrazer\\AppData\\Local\\Temp\\pip-build-s2i8soug\\wordcloud\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\akrazer\AppData\Local\Temp\tmpzi7o5i7jpip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.6
  creating build\lib.win-amd64-3.6\wordcloud
  copying wordcloud\color_from_image.py -> build\lib.win-amd64-3.6\wordcloud
  copying wordcloud\tokenization.py -> build\lib.win-amd64-3.6\wordcloud
  copying wordcloud\wordcloud.py -> build\lib.win-amd64-3.6\wordcloud
  copying wordcloud\wordcloud_cli.py -> b

  Failed building wheel for wordcloud
Command "C:\Users\akrazer\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\akrazer\\AppData\\Local\\Temp\\pip-build-s2i8soug\\wordcloud\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\akrazer\AppData\Local\Temp\pip-4xs7gqel-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\akrazer\AppData\Local\Temp\pip-build-s2i8soug\wordcloud\


  Using cached tqdm-4.19.5-py2.py3-none-any.whl


Подгружаем данные:

In [7]:
import pandas as pd

train = pd.read_csv("labeledTrainData.tsv", header=0,
                    delimiter="\t", quoting=3)

In [8]:
train['review'].iloc[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

Видно, что в данных присутствуют элементы html.

- избавимся от элементов html  с помощью модуля BeautifulSoup: 
    http://nullege.com/codes/search/bs4.BeautifulSoup.getText
- оставим только буквы
- приведем все к нижнему регистру
- уберем стоп слова

In [10]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

AttributeError: module 'nltk' has no attribute 'data'

In [39]:
def review_to_words(raw_review):
    # your code is hear
    # 1. bs
    bs = BeautifulSoup(raw_review)
    review_text = bs.get_text()
    # 2. regular expression
    rege = re.compile('\w+')
    letters_only = rege.findall(review_text)
    # 3. to lower
    words = [word.lower() for word in letters_only]
    # 4. drop stopwords
    sw = stopwords.words('english')
    words = [w for w in words if w.lower() not in sw]    
    return words 

In [40]:
review_to_words(train['review'].iloc[0])

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 '20',
 'minutes',
 'excluding',
 'smooth',

Примените реализованную функцию к каждому предожению train

In [50]:
from tqdm import tqdm

#num_reviews = train["review"].size 
num_reviews = 1000

clean_train_reviews = []

for i in tqdm(range(num_reviews)):
    clean_train_reviews.append(review_to_words(train["review"][i]))

  0%|                                                 | 0/1000 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|█████████████████████████████████████| 1000/1000 [00:02<00:00, 362.98it/s]


Обучите word2vec

In [42]:
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec

# подаем в обучение clean_train_reviews
model = word2vec.Word2Vec(clean_train_reviews, workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

model.init_sims(replace=True)

# модель можно сохранить
# model_name = ''
# model.save(model_name)

Теперь можно воспользjваться функциями поиска похожих / непохожих слов на данное
- model.wv.most_similar
- model.wv.doesnt_match
- etc

In [43]:
# здесь можно посмотреть все слова, к которым мы имеем вектора
#model.wv.vocab

In [44]:
model.wv.most_similar('mother')

[('father', 0.9997919797897339),
 ('next', 0.9997912645339966),
 ('human', 0.9997851848602295),
 ('friend', 0.9997850656509399),
 ('called', 0.9997849464416504),
 ('course', 0.9997843503952026),
 ('son', 0.9997842907905579),
 ('name', 0.9997837543487549),
 ('becomes', 0.9997789859771729),
 ('woman', 0.9997783899307251)]

In [45]:
model.wv.doesnt_match("man woman child".split())

'child'

In [46]:
model.wv.doesnt_match(["sun", "moon", "planet", "earth"])

'earth'

In [47]:
model.wv.most_similar("girl")

[('next', 0.9998335838317871),
 ('set', 0.9998206496238708),
 ('tries', 0.9998139142990112),
 ('children', 0.9998130202293396),
 ('looking', 0.9998107552528381),
 ('goes', 0.9998093843460083),
 ('lead', 0.9998011589050293),
 ('supposed', 0.9997996687889099),
 ('given', 0.9997984170913696),
 ('always', 0.9997984170913696)]

In [48]:
model.wv.most_similar("horror")

[('early', 0.9997045397758484),
 ('based', 0.9997020363807678),
 ('genre', 0.9997010231018066),
 ('bit', 0.9996837377548218),
 ('sense', 0.9996798038482666),
 ('pretty', 0.9996776580810547),
 ('fans', 0.9996731281280518),
 ('kind', 0.9996672868728638),
 ('british', 0.9996665716171265),
 ('dialogue', 0.9996662139892578)]

Теперь выделите top 100 самых частотных слов и получите для них только что обученные вектора

In [68]:
review_to_words(train['review'].iloc[0])

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 '20',
 'minutes',
 'excluding',
 'smooth',

In [1]:
from nltk import FreqDist
fd = FreqDist()
for i in range(len(train["review"])):
    text = review_to_words(train["review"].iloc[i])
    for j in range(len(text)):
        if j in fd:
            fd[text[j]] += 1
        else:
            fd[text[j]] = 0
fd

# your code is hear

#top_words = # your code is hear

KeyboardInterrupt: 

In [ ]:
top_words_vec = # your code is hear

Визуализируем вектора. Для этого воспользуемся методом снижения размерности t-sne

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = # your code is hear

In [ ]:
print(top_words_vec.shape)
print(top_words_tsne.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

x = []
y = []
for value in top_words_tsne:
    x.append(value[0])
    y.append(value[1])

plt.figure(figsize=(16, 16)) 
for i in range(100):
    plt.scatter(x[i],y[i])
    plt.annotate(top_words[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()

Теперь нарисуйте облако слов

In [ ]:
from wordcloud import *
word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()